<a href="https://colab.research.google.com/github/yashsakhuja/Stocks-Data-Downloader/blob/main/Getting%20Started%20with%20YFinance_API%20in%20Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and Importing Packages


In [ ]:
! pip install -q gspread oauth2client yfinance

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Getting the Parameters

In [ ]:
# List of tickers
tickers = ['LICI.NS', 'INFY.NS', 'HDFCBANK.NS','WIPRO.NS','TATAMOTORS.NS','BHARTIARTL.BO','TCS.NS']

# Number of years of historical data
nyears = 5

# Calculate the start and end dates
end_date = datetime.today()
start_date = end_date - timedelta(days=nyears * 365)

# API Call, Getting the Data and Consolidating

In [ ]:
# Download historical data
data = yf.download(tickers, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

In [ ]:
data.head()

In [ ]:
# Extract the 'Close' prices
all_closing_prices = data['Close']

In [ ]:
all_closing_prices.head()

In [ ]:
# Transform data to long format
final_data = all_closing_prices.reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Close')

In [ ]:
final_data.tail()

# **Additional**: Updating Google Sheets File

This is a additional section not covered in the blog but if you want to build your own google sheets updating mechanism you could build one with the given code snippet by updating the credentials path and your google sheet file name (and worksheet name if required)

Before you run these set of codes an essential pre-requisites are:

## **Initial setup for private spreadsheet and/or CRUD mode**

> **Enable API Access for a Project**


1.   Head to Google Developers Console and create a new project (or select the one you already have).

2.   In the box labeled “Search for APIs and Services”, search for “Google Drive API” and enable it.

3.   In the box labeled “Search for APIs and Services”, search for “Google Sheets API” and enable it.



> **Using Service Account**



1.   Enable API Access for a Project if you haven’t done it yet.

      Go to “APIs & Services > Credentials” and choose “Create credentials > Service account key”.

      Fill out the form
      Click “Create” and “Done”.

2.   Press “Manage service accounts” above Service Accounts.


3.   Press on ⋮ near recently created service account and select “Manage keys” and then click on “ADD KEY > Create new key”.

4.   Select JSON key type and press “Create”.
You will automatically download a JSON file with credentials.




> Finally, Upload this json key file on your gdrive and use the file location as reference for credentials











In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
#Mounting my Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Initializing the scope
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]


In [ ]:
# Path to the credentials JSON file in Google Drive
creds_path = '/content/drive/My Drive/Your_Key_Json_Filename.json'  # Path for the Google Service Json Key stored in Drive

In [ ]:
# initialize the gspread client with scope and credential
creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
client = gspread.authorize(creds)

In [ ]:
# Open the Google Sheet
sheet = client.open("Your_Google_Sheet_Name").sheet1  # Open the first sheet of the spreadsheet

# Clear the sheet before writing new data
sheet.clear()

In [ ]:
# Convert DataFrame to list of lists, ensuring all values are serializable
# Else it give you error that the data is not serializable
data_to_insert = [final_data.columns.tolist()] + final_data.astype(str).values.tolist()

In [ ]:
# Update the sheet with new data
sheet.update(data_to_insert)

print("Data successfully exported to Google Sheets!")